# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis y visualización de datos

### Mentoría 16 - Trabajo práctico 1

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---

## Librerías

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
pd.options.display.float_format = '{:,.2f}'.format

## Dataset

In [ ]:
url = '/home/usuario/muestra_diplodatos_ventas_2023.csv'
ventas = pd.read_csv(url)

---
# Paso 1: Carga y exploración inicial de datos

1.	Cargar los datos.
2.	Explora la estructura de los datos: número de observaciones, variables disponibles, intervalo de tiempo entre observaciones, etc.
3.	Observa las primeras y últimas filas de los datos para tener una idea general de su contenido.
4.	Verifica si hay valores faltantes o atípicos en los datos y propongan cómo manejarlos.


In [ ]:
lista_columnas = list(ventas.columns)
print(len(lista_columnas), 'variables:')
lista_columnas

Significado tentativo:
*   **ID_VENDEDOR**: Clave identificación vendedor
*   **INSCRIPCION**: ???
*   **AÑO**: Año (2019 a 2022)
*   **MES**: 	Mes
*   **CATEGORIA**: Descripción del Código de Actividad Codiguero NAES Año 2018 y siguientes
*   **DEPOSITO**: ???
*   **DESCRIPCION_CATEGORIA**: Subrubro definido por la DGR (Dato Interno, no declarado por el contribuyente).
*   **TOTAL_VENTAS**: Monto total de ventas (base imponible)
*   **PORCENTAJE_COMISION_EMPRESA**: Comisión por las ventas en la plataforma (alícuota)
*   **COMISION_EMPRESA**: Comisión por las ventas en la plataforma (valor)
*   **TRATAMIENTO_FISCAL**: Indica qué tratamiento fiscal se le da a la operación (o vendedor/comprador?), especialmente respecto a la alícuota a cobrarle.
*   **DESC_TRATAMIENTO_FISCAL**: Categoriza los valores de "TRATAMIENTO_FISCAL"
*   **TRATAMIENTO_DIFERNCIAL**: Indica el artículo de alguna reglamentación aplicado para dar tratamiento fiscal especial
*   **CM04**: Puede marcar las operaciones bajo Convenio Multilateral (CM) o indicar que se trata de un vendedor inscripto en el CM. Ver.
*   **CATEGORIA (Ajustado)**: Variable "CATEGORIA" limpia (se redujo el número de valores posibles)
*   **SUB-CATEGORIA**: Variable "DESCRIPCION_CATEGORIA" limpia (se redujo el número de valores posibles)
*   **NOMBRE**: Nombre del vendedor? Del comprador?
*   **OMEGA**: Marca para contribuyente de interés fiscal especial?? (Todos tienen valor "1")
*   **MODELO**: Marca para vendedor modelo

In [ ]:
ventas.dtypes

In [ ]:
ventas.describe()

In [ ]:
msno.bar(ventas,figsize=(12, 3), sort="ascending",fontsize=12, color='gray')

In [ ]:
recuentos = {}

for columna in ventas.columns:
    recuento = ventas[columna].value_counts()
    recuentos[columna] = recuento
    print(f"Recuento de categorías para la columna '{columna}':")
    print(recuento)
    print()
    print()
    print()



En base a lo visto hasta aquí, se podría prescindir de las variables "CATEGORIA" y "DESCRIPCION_CATEGORIA" (superadas por "CATEGORIA (Ajustado)" y "SUB-CATEGORIA", respectivamente).

También se podría prescindir de la variable "OMEGA", por no aportar información relevante (es constante).

Al igual que con "CATEGORIA (Ajustado)"

In [ ]:
ventas_clean = ventas.drop(['CATEGORIA', 'DESCRIPCION_CATEGORIA', 'OMEGA', 'CATEGORIA (Ajustado)'], axis=1).copy()
ventas_clean.head()

In [ ]:
ventas_clean['FECHA'] = pd.to_datetime(ventas_clean['MES'].astype(str) + '-' + ventas_clean['AÑO'].astype(str), format='%m-%Y').dt.to_period('M')
ventas_clean.head()

In [ ]:
ventas_clean['FECHA'].dtypes

*   Qué representa cada caso? Se trata de ventas, de ventas por mes, vendedores por mes?

In [ ]:
# Vamos a eliminar duplicados por ID_VENDEDOR y FECHA
prueba1 = ventas_clean.drop_duplicates(subset=['ID_VENDEDOR', 'FECHA']).copy()
print(len(prueba1))
print(len(ventas_clean))

In [ ]:
# Lo anterior implica que hay más de una observación para cada mes por vendedor. Probablemente cada caso represente operaciones de ventas individuales.

*   "ID_VENDEDOR" se corresponde con "NOMBRE"?

In [ ]:
# Cuántos ID_VENDEDOR únicos hay?
prueba2 = ventas_clean.drop_duplicates(subset='ID_VENDEDOR').copy()
print(len(prueba2))

In [ ]:
# Cuántos NOMBRE únicos hay?
prueba3 = ventas_clean.drop_duplicates(subset='NOMBRE').copy()
print(len(prueba3))

In [ ]:
# Implica que hay algunos ID_VENDEDOR que tienen el mismo NOMBRE

In [ ]:
# Quizas se pueda eliminar la variabla NOMBRE

*   Cómo se relacionan TRATAMIENTO_FISCAL,	DESC_TRATAMIENTO_FISCAL y	TRATAMIENTO_DIFERNCIAL?


In [ ]:
msno.bar(ventas_clean[['TRATAMIENTO_FISCAL', 'DESC_TRATAMIENTO_FISCAL', 'TRATAMIENTO_DIFERNCIAL']],figsize=(4, 3), sort="ascending",fontsize=12, color='gray')

In [ ]:
# La variable con más casos es TRATAMIENTO_FISCAL

In [ ]:
ventas_clean['TRATAMIENTO_FISCAL'].value_counts()

In [ ]:
# Parece que asume valores enteros, floats y strings.
# Pero acá se ve que los que parecen enteros, en realidad son strings
ventas_clean['TRATAMIENTO_FISCAL'].unique()

In [ ]:
# En primera instancia, se podrían forzar los floats hacia strings.
ventas_clean['TRATAMIENTO_FISCAL'] = ventas_clean['TRATAMIENTO_FISCAL'].replace({0.0: '0', 3.0: '3', 2.0: '2', 1.0: '1'})

In [ ]:
ventas_clean['TRATAMIENTO_FISCAL'].value_counts()

In [ ]:
# Ahora veamos para qué valores de TRATAMIENTO FISCAL, suelen aparecer valores de DESC_TRATAMIENTO_FISCAL

In [ ]:
# Este paso lo agrego porque sino el crosstab siguiente no me muestra los NaN
prueba5 = ventas_clean.copy()
prueba5['TRATAMIENTO_FISCAL'] = prueba5['TRATAMIENTO_FISCAL'].fillna('')
prueba5['DESC_TRATAMIENTO_FISCAL'] = prueba5['DESC_TRATAMIENTO_FISCAL'].fillna('')

In [ ]:
pd.crosstab(prueba5['TRATAMIENTO_FISCAL'], prueba5['DESC_TRATAMIENTO_FISCAL'], dropna=False)

In [ ]:
# Los valores de DESC_TRATAMIENTO_FISCAL, solo aparecen cuando TRATAMIENTO_FISCAL asume valores 0, 1, 2, 3.
# A su vez, hay correspondencia entre 0 y Normal, 1 y Exento/Desgravado, 2 y Minorista, 3 y Otro Tratamiento Fiscal

In [ ]:
# Se podría eliminar DESC_TRATAMIENTO_FISCAL?

In [ ]:
# Ahora veamos para qué valores de TRATAMIENTO FISCAL, suelen aparecer valores de TRATAMIENTO_DIFERNCIAL

In [ ]:
# Este paso lo agrego porque sino el crosstab siguiente no me muestra los NaN
prueba5['TRATAMIENTO_DIFERNCIAL'] = prueba5['TRATAMIENTO_DIFERNCIAL'].fillna('')

In [ ]:
pd.crosstab(prueba5['TRATAMIENTO_FISCAL'], prueba5['TRATAMIENTO_DIFERNCIAL'], dropna=False)

In [ ]:
# No hay un patrón claro

*   Ventas, comisión y % de comisión están relacionadas entre sí con alguna fórmula?

In [ ]:
ventas_clean[['TOTAL_VENTAS','PORCENTAJE_COMISION_EMPRESA','COMISION_EMPRESA']].describe()

In [ ]:
# TOTAL_VENTAS tiene valores negativos. Cuandos?
len(ventas_clean[ventas_clean['TOTAL_VENTAS'] < 0])

In [ ]:
ventas_clean[ventas_clean['TOTAL_VENTAS'] < 0]['TOTAL_VENTAS'].describe()

In [ ]:
# Hay al menos un valor extremo muy grande.
# Debido a que son pocos casos, podríamos reemplazar estos valores por vacíos (en caso que estén mal).

In [ ]:
# Se observa que COMISION_EMPRESA también tiene valores negativos en estos casos. También sería necesario reemplazar estos valores por vacíos
ventas_clean[ventas_clean['TOTAL_VENTAS'] < 0]['COMISION_EMPRESA'].describe()

In [ ]:
# No pareciera haber valores extremos en PORCENTAJE_COMISION_EMPRESA
ventas_clean[ventas_clean['TOTAL_VENTAS'] < 0]['PORCENTAJE_COMISION_EMPRESA'].describe()

In [ ]:
# ventas_clean.loc[ventas_clean['TOTAL_VENTAS'] < 0, 'TOTAL_VENTAS'] = np.nan
# ventas_clean.loc[np.isnan(ventas_clean['TOTAL_VENTAS']), 'COMISION_EMPRESA'] = np.nan
# ventas_clean.loc[np.isnan(ventas_clean['TOTAL_VENTAS']), 'PORCENTAJE_COMISION_EMPRESA'] = np.nan
# REVISAR (VER OTRA ALTERNATIVA AL nan)

In [ ]:
ventas_clean[['TOTAL_VENTAS','PORCENTAJE_COMISION_EMPRESA','COMISION_EMPRESA']].describe()

In [ ]:
# Crear una figura con tres subplots
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

# Histograma para TOTAL_VENTAS
axs[0].hist(ventas_clean['TOTAL_VENTAS'], bins=25, edgecolor='black')
axs[0].set_xlabel('TOTAL_VENTAS')
axs[0].set_ylabel('Frecuencia')
axs[0].set_title('TOTAL_VENTAS')

# Histograma para COMISION_EMPRESA
axs[1].hist(ventas_clean['COMISION_EMPRESA'], bins=25, edgecolor='black')
axs[1].set_xlabel('COMISION_EMPRESA')
axs[1].set_ylabel('Frecuencia')
axs[1].set_title('COMISION_EMPRESA')

# Histograma para PORCENTAJE_COMISION_EMPRESA
axs[2].hist(ventas_clean['PORCENTAJE_COMISION_EMPRESA'], bins=25, edgecolor='black')
axs[2].set_xlabel('PORCENTAJE_COMISION_EMPRESA')
axs[2].set_ylabel('Frecuencia')
axs[2].set_title('PORCENTAJE_COMISION_EMPRESA')

# Ajustar los espacios entre subplots
fig.tight_layout()

# Mostrar los histogramas
plt.show()

In [ ]:
# Crear la figura y los subplots
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

# Boxplot para 'TOTAL_VENTAS'
axs[0].boxplot(ventas_clean['TOTAL_VENTAS'].dropna())
axs[0].set_title('Boxplot de TOTAL_VENTAS')

# Boxplot para 'COMISION_EMPRESA'
axs[1].boxplot(ventas_clean['COMISION_EMPRESA'].dropna())
axs[1].set_title('Boxplot de COMISION_EMPRESA')

# Boxplot para 'PORCENTAJE_COMISION_EMPRESA'
axs[2].boxplot(ventas_clean['PORCENTAJE_COMISION_EMPRESA'].dropna())
axs[2].set_title('Boxplot de PORCENTAJE_COMISION_EMPRESA')

# Ajustar los espacios entre subplots
plt.tight_layout()

# Mostrar los gráficos
plt.show()

In [ ]:
# Deberíamos hacer algo con los valores negativos, los valores demasiado altos (y con los 0?)

In [ ]:
(ventas_clean['TOTAL_VENTAS'] == 0).sum()

In [ ]:
(ventas_clean['COMISION_EMPRESA'] == 0).sum()

In [ ]:
plt.scatter(ventas_clean['COMISION_EMPRESA'], ventas_clean['TOTAL_VENTAS'])
plt.xlabel('COMISION_EMPRESA')
plt.ylabel('TOTAL_VENTAS')
plt.title('COMISION_EMPRESA vs TOTAL_VENTAS')
plt.show()

In [ ]:
plt.scatter(ventas_clean['TOTAL_VENTAS'], ventas_clean['PORCENTAJE_COMISION_EMPRESA'])
plt.xlabel('TOTAL_VENTAS')
plt.ylabel('PORCENTAJE_COMISION_EMPRESA')
plt.title('TOTAL_VENTAS vs PORCENTAJE_COMISION_EMPRESA')
plt.show()

In [ ]:
plt.scatter(ventas_clean['COMISION_EMPRESA'], ventas_clean['PORCENTAJE_COMISION_EMPRESA'])
plt.xlabel('COMISION_EMPRESA')
plt.ylabel('PORCENTAJE_COMISION_EMPRESA')
plt.title('COMISION_EMPRESA vs PORCENTAJE_COMISION_EMPRESA')
plt.show()

In [ ]:
# Ver de  armar un pairplot o jointplot?

In [ ]:
# Probemos con verificar la siguiente fórmula:
ventas_clean['CALCULADA'] = ventas_clean['TOTAL_VENTAS'] * ventas_clean['PORCENTAJE_COMISION_EMPRESA']

In [ ]:
# Cuándo hay coincidencia exacta?
sum(abs(ventas_clean['COMISION_EMPRESA'] - ventas_clean['CALCULADA'] == 0))

In [ ]:
# Es decir, sólo en 191 mil casos la fórmula se cumple exactamente.

In [ ]:
# Probemos con algún margen de error

In [ ]:
sum(abs(ventas_clean['COMISION_EMPRESA'] - ventas_clean['CALCULADA']) <= 0.01)

In [ ]:
for x in [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1]:
  print('Para', x, 'de diferencia, tenemos', sum(abs(ventas_clean['COMISION_EMPRESA'] - ventas_clean['CALCULADA']) <= x), 'igualdades')

In [ ]:
# Para 261 mil casos (61%) la fórmula se verifica, con precisión menor a 1 entero.

In [ ]:
# Grafiquemos la distribución de la diferencia

diferencia = ventas_clean['COMISION_EMPRESA'] - ventas_clean['CALCULADA']
plt.boxplot(diferencia.dropna())
plt.xlabel('Diferencia')
plt.ylabel('Valor')
plt.title('Box Plot de la diferencia entre COMISION_EMPRESA y CALCULADA')
plt.show()

In [ ]:
# Surge otra pregunta: habrá algun caso que la diferencia es positiva?
diferencia.describe()

In [ ]:
plt.scatter(ventas_clean['COMISION_EMPRESA'], ventas_clean['CALCULADA'])
plt.xlabel('COMISION_EMPRESA')
plt.ylabel('CALCULADA')
plt.title('COMISION_EMPRESA vs CALCULADA')
plt.show()

In [ ]:
# Poniendo foco en los casos en que COMISION_EMPRESA es mayor que CALCULADA:
plt.scatter(ventas_clean[ventas_clean['COMISION_EMPRESA'] > ventas_clean['CALCULADA']]['COMISION_EMPRESA'], ventas_clean[ventas_clean['COMISION_EMPRESA'] > ventas_clean['CALCULADA']]['CALCULADA'])
plt.xlabel('COMISION_EMPRESA')
plt.ylabel('CALCULADA')
plt.title('COMISION_EMPRESA vs CALCULADA')
plt.show()

In [ ]:
len(ventas_clean[ventas_clean['COMISION_EMPRESA'] > ventas_clean['CALCULADA']])

In [ ]:
# Lo que figura en la variable COMISION_EMPRESA siempre es menor o igual que lo que surgiría de multiplicar TOTAL_VENTAS y PORCENTAJE_COMISION_EMPRESA

## Paso 2: Análisis estadístico descriptivo

1.	Calcula estadísticas descriptivas básicas de la serie temporal, como la media, mediana, desviación estándar, mínimo y máximo.


In [ ]:
ventas['TOTAL_VENTAS'].describe()

2.	Analiza la tendencia central y la dispersión de los datos. ¿Existen valores atípicos o extremos? ¿Cómo podrían afectar el análisis posterior?


In [ ]:
#PROPONER ALGUN PERCENTIL. Y PONER EL NRO ABSOLUTO EN CANTIDAD DE REGISSTRO

In [ ]:
plt.boxplot(ventas_clean['TOTAL_VENTAS'].dropna())
plt.title('TOTAL_VENTAS')
plt.show()

In [ ]:
# Los valores negativos y los 0 sesgan hacia abajo la media
# Los valores extremos positivos sesgan hacia arriba

In [ ]:
# Negativos
len(ventas_clean[ventas_clean['TOTAL_VENTAS'] < 0])

In [ ]:
# Ceros
(ventas_clean['TOTAL_VENTAS'] == 0).sum()

In [ ]:
# Extremos positivos
percentil_95 = np.percentile(ventas_clean['TOTAL_VENTAS'], 95)
len(ventas_clean[ventas_clean['TOTAL_VENTAS'] >= percentil_95])

In [ ]:
# Al eliminar los extremos positivos y negativos
plt.boxplot(ventas_clean[(ventas_clean['TOTAL_VENTAS'] > 0) & (ventas_clean['TOTAL_VENTAS'] < percentil_95)]['TOTAL_VENTAS'])
plt.title('TOTAL_VENTAS')
plt.show()

In [ ]:
#Agregar histograma

3.	Grafica la serie temporal en un gráfico de líneas para visualizar la evolución de los datos a lo largo del tiempo.

In [ ]:
prueba6 = ventas_clean[['FECHA','TOTAL_VENTAS']].copy()

In [ ]:
prueba6['FECHA'].dtype

In [ ]:
prueba6['TOTAL_VENTAS'].dtype

In [ ]:
prueba6_agregado = prueba6.groupby('FECHA')['TOTAL_VENTAS'].sum().reset_index()

plt.figure(figsize=(16, 3))

# Graficar el gráfico de líneas
plt.plot(prueba6_agregado['FECHA'].astype(str), prueba6_agregado['TOTAL_VENTAS'])

# Personalizar el gráfico
plt.title('Total de ventas')
plt.xlabel('Fecha')
plt.ylabel('Ventas')

# Rotar las etiquetas del eje X para mayor legibilidad (opcional)
plt.xticks(rotation=45)

# Mostrar el gráfico
plt.show()

In [ ]:
len(prueba6_agregado)

In [ ]:
# GRAFICAR POR AÑO. LA TENDENCIA DE MESES. PARA VER ESTACIONALIDAD.

In [ ]:
# Se observa el efecto de la inflación y pareciera haber picos en noviembre/diciembre

4.	Calcula y grafica la función de autocorrelación para identificar posibles patrones de autocorrelación en los datos. (Es un término estadístico que se utiliza para describir la presencia o ausencia de correlación en los datos de las series temporales, indicando, si las observaciones pasadas influyen en las actuales.)

In [ ]:
# Calcular la función de autocorrelación
acf = plot_acf(prueba6_agregado['TOTAL_VENTAS'], lags=12)

# Graficar la función de autocorrelación
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function')
plt.show()

In [ ]:
# Hay correlación estadísticamente significativa para el lag 1, 2 y 3.
# Lo que implica que observaciones pasadas influyen en las observaciones actuales de la serie temporal.
# De manera positiva